### 1.0) Introdução

Aqui entramos em uma parta muito importante. Até então, tudo que fizemos foi nos conectar ao API do chat para que ele nos desse respostas de acordo com o que era perguntado. Bem, isso não muda muita coisa em relação ao que fazemos na própria interface do chat GPT caso nosso propósito fosse fazer perguntas e obter respostas.

Portanto, iremos mostrar ao chat GPT que ele além de sua ferramenta incrível de geração de texto, também poderá acessar a base de dados ou funções que nós mesmo criarmos com informações específicas.


### 1.1) Function Callings (CHAMANDO FUNÇÕES)

Para ensinar sobre, nada melhor do que um exemplo.

O exemplo consiste no seguinte. Queremos que o chat nos diga informações sobre o tempo em determinadas regiões. Porém, ele só poderá acessar essas informações de uma função criada. Para facilitar nosso estudo, essa função terá apenas dois parâmetros sendo eles a localidade e unidade desejada. No entanto, em níveis mais avançados, podemos conectar a sites ou base de dados consolidadas.

In [2]:
# importando as bibliotecas

import json
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
client = openai.Client()

##### 1º) Definindo a nossa função que retorna a temperatura

In [3]:
def obter_temperatura_atual(local, unidade = 'celcius'):
    if 'são paulo'in local.lower():
        return json.dumps(
            {'local':'São Paulo', 'temperatura': '32', 'unidade': unidade}
        ) ### o comando "json.dumps" serve simplesmente para transformar esse dicionário em uma string. Até porque é uma llm.
    elif "porto alegre" in local.lower():
        return json.dumps(
            {'local': "Porto Alegre", 'temperatura':'25', 'unidade':unidade}
        )
    elif "rio de janeiro" in local.lower():
        return json.dumps(
            {'local': "Rio de Janeiro", 'temperatura':'35', 'unidade':unidade}
        )
    else: ### esse else seria caso não fosse nenhum dos outros 3. Logo, daria uma resposta desconhecida
        return json.dumps(
            {'local': local, 'temperatura':'unknown'}
        )

##### 2º) Como anexar a função à resposta do chat?

Agora, quando formos rodar o código que o chat nos gera uma resposta, iremos colocar também o parâmetro "tools", que indica as ferramentas que ele pode ter acesso ao gerar a resposta. Irei colocar o código do tools, mas antes veja o código da resposta!

A variável tools será uma lista de dicionários. Cada dicionário será uma tool específica (pode ter mais de uma). Nela, iremos especificar algumas coisas como o tipo de tool (no caso uma função que retorna a temperatura)

a descrição que a tool possui, os parâmetros, o que faz etc...

In [5]:
tools = [
    {
    'type':'function',
    'function':{
        "name":'obter_temperatura_atual',
        "description":"Obtém a temperatura atual em um dada cidade",
        'parameters':{
            'type':'object', ##sempre deixar type objects
            'properties':{
                'local':{
                    'type':'string',
                    'descripition': "O nome da cidade. Ex: São Paulo",
                },
                'unidade':{
                    'type':'string',
                    'enum': ["celsius", 'fahrenheit'] #limita as possibilidades para essas 2
                }
            },
            "required":['local'] # indica quais parâmetros são necessários
        }
    }
}
]

In [1]:
####### CÓDIGO DA RESPOSTA

### Caso o nosso tools tivesse mais de uma função, é comum criarmos o seguinte dicionário que correlaciona o nome da função com a própria função

funcoes_disponiveis = {
    'obter_temperatura_atual': obter_temperatura_atual
}


mensagens = [
    {'role':'user', 'content':'Qual é a temperatura em São Paulo e Porto Alegre?'}
]


resposta = client.chat.completions.create(
    model = 'gpt-3.5-turbo-0125',
    messages = mensagens,
    tools = tools,
    tool_choice = 'auto'
) ## tool_choice é como o chat irá escolher essa ferramenta. Quando colocamos "auto" ele pode escolher de acordo como achar melhor. Mas podemos obriga-lo a escolher uma tool específica.

NameError: name 'obter_temperatura_atual' is not defined

Veja agora o que é a resposta dele:

In [9]:
resposta ### Repare que quando chegamos em "content" aparece None. E em "tool_calls" ele nos informa quais parâmetros devemos rodar na função para que ele obtenha a resposta desejada.

ChatCompletion(id='chatcmpl-AtiDCqTd5hrZrDiXXsChPF9D0XogY', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_xvzrpEG77oCvYnj0gll5AAGJ', function=Function(arguments='{"local": "São Paulo", "unidade": "celsius"}', name='obter_temperatura_atual'), type='function'), ChatCompletionMessageToolCall(id='call_AuI6e2THupTeBGFpiYQBTyeB', function=Function(arguments='{"local": "Porto Alegre", "unidade": "celsius"}', name='obter_temperatura_atual'), type='function')]))], created=1737841354, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=70, prompt_tokens=79, total_tokens=149, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_to

Se você reparar, o atributo tool_calls é uma lista com 2 resultados.

In [10]:
resposta.choices[0].message.tool_calls[0] ### primeiro elemento da lista

ChatCompletionMessageToolCall(id='call_xvzrpEG77oCvYnj0gll5AAGJ', function=Function(arguments='{"local": "São Paulo", "unidade": "celsius"}', name='obter_temperatura_atual'), type='function')

In [11]:
resposta.choices[0].message.tool_calls[1] ### segundo elemento da lista

ChatCompletionMessageToolCall(id='call_AuI6e2THupTeBGFpiYQBTyeB', function=Function(arguments='{"local": "Porto Alegre", "unidade": "celsius"}', name='obter_temperatura_atual'), type='function')

##### 3º) Como rodar a função e fornecer a resposta para o chat

Vamos recapitular um pouco o que foi feito até então. Nós fizemos uma pergunta ao chat referente a temperatura de dois lugares. E colocamos como parâmetro das respostas o tool. Esse tool serve para indicar que o API do chat GPT tem acesso a uma lista de ferramentas. No nosso caso essa lista tinha apenas uma função que chamamos de "obter a temperatura atual". Logo, quando rodamos a resposta, vimos acima que ele não gerou nenhum "content" como resposta. Ao invés disso, ele falou que a resposta se encontra ao rodar a função "obter_temperatura_atual com os parâmetros específicos que ele mesmo recomendou.

Logo, o que precisamos fazer nesse momento, é rodar as funções, obter a resposta e enviar para ele. Ok, mas como enviar para ele? Lembra lá no início do curso, aquela lista de mensagens registradas que colocávamos role = ['user' ou 'assistent']. Então, devemos registrar a resposta da função como "role":"tool". Isso é, ele vai entender que naquele histórico de conversa, em algum momento ele teve que rodar a função para o assistente gerar a resposta!

In [13]:
mensagem_resp = resposta.choices[0].message # isso representa a mensagem que o chat deu, leia em resposta
tool_calls = mensagem_resp.tool_calls # representa a lista com as duas chamadas de função mostradas acima

if tool_calls: ### isso serve para qualquer caso em que tool call não for NONE. Por que se for none, quer dizer que o chat não quer que chame as funções para obter as respostas.
    mensagens.append(mensagem_resp) # colocando no registro de conversa do chat a informação da resposta que ele deu. Em sumo, 0 content e apenas a chamada das funçoes
    for tool_call in tool_calls: # acessando cada um dos 2 tool_calls gerados
        function_name = tool_call.function.name
        function_to_call = funcoes_disponiveis[function_name]
        function_args = json.loads(tool_call.function.arguments) # o json.loads é uma forma de trasnformar o dicionário que estava escrito como string em apenas o dicionário. É o contrário do que foi feito la em cima com o json.dumps
        function_response = function_to_call(local = function_args['local'], unidade = function_args['unidade']) # também pode usar o comando .get para selecionar a chave do dicionário; Ex: function_args.get('local')

        mensagens.append(
            {
                'tool_call_id': tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response
            }
        )

In [14]:
mensagens

[{'role': 'user',
  'content': 'Qual é a temperatura em São Paulo e Porto Alegre?'},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_FeduKsMJ28y6nCdyc8wKUXQw', function=Function(arguments='{"local": "São Paulo", "unidade": "celsius"}', name='obter_temperatura_atual'), type='function'), ChatCompletionMessageToolCall(id='call_5lVKjUe2s84dBH1wXAr6tk53', function=Function(arguments='{"local": "Porto Alegre", "unidade": "celsius"}', name='obter_temperatura_atual'), type='function')]),
 {'tool_call_id': 'call_FeduKsMJ28y6nCdyc8wKUXQw',
  'role': 'tool',
  'name': 'obter_temperatura_atual',
  'content': '{"local": "S\\u00e3o Paulo", "temperatura": "32", "unidade": "celsius"}'},
 {'tool_call_id': 'call_5lVKjUe2s84dBH1wXAr6tk53',
  'role': 'tool',
  'name': 'obter_temperatura_atual',
  'content': '{"local": "Porto Alegre", "temperatura": "25", "unidade": "celsius"}'}]

##### 4º) Gerando a resposta final

In [15]:
resposta_final = client.chat.completions.create(
    model = 'gpt-3.5-turbo-0125',
    messages = mensagens
)

In [16]:
resposta_final

ChatCompletion(id='chatcmpl-AtirMn9TxX0Ib10C4RNY9MV7d1mFt', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='A temperatura atual em São Paulo é de 32°C e em Porto Alegre é de 25°C.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1737843844, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=24, prompt_tokens=155, total_tokens=179, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

Repare agora que o "content" tem resultado. Vamos buscá-lo.

In [17]:
resposta_final.choices[0].message.content

'A temperatura atual em São Paulo é de 32°C e em Porto Alegre é de 25°C.'